In [120]:
import numpy as np
import  cv2

In [121]:
image  = cv2.imread("Fig_340.tif")
image_blur = image.copy()

In [122]:
k  = 3

In [123]:

def shift(dx, dy, image):
    x_start = max(0, -dx)
    y_start = max(0, -dy)
    
    x_end = min(image.shape[0], image.shape[0] - dx)
    y_end = min(image.shape[1], image.shape[1] - dy)
    image = image[x_start:x_end, y_start:y_end]
    #padding de zeros depois que translada a imagem

    if(dx<0 and dy<0) :
        image = np.pad(image, ((0,-dx),(0,-dy)), mode='constant', constant_values=0)
    elif(dx<0 and dy>=0) :
        image = np.pad(image, ((0,-dx),(dy,0)), mode='constant', constant_values=0)
    elif(dx>=0 and dy<0) :
        image = np.pad(image, ((dx,0),(0,-dy)), mode='constant', constant_values=0)
    elif(dx>=0 and dy>=0) :
        image = np.pad(image, ((dx,0),(dy,0)), mode='constant', constant_values=0)
  
  
    return image

In [124]:
def convolution(mask, image):
    mask = np.flipud(np.fliplr(mask)) # rebate a mascara
    # tam_pad = (mask.shape[0]-1)//2
    # image_pad = np.pad(image,tam_pad , 'constant') # zero-padding
    # image=image_pad
    image_conv = np.zeros((image.shape[0],image.shape[1]))

    #varrendo a mascara(como mostrado em sala)
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            auxiliar_matrix = shift(i-(mask.shape[0]-1//2),j-(mask.shape[0]-1//2),image)#deslocamento
            auxiliar_matrix_multiply_by_weight =  auxiliar_matrix*mask[i][j]
            image_conv = image_conv + auxiliar_matrix_multiply_by_weight
    return image_conv

        

In [125]:
def average_filter(mask):
    total = np.sum(mask)
    result_mask = mask/total
    return result_mask


In [126]:
mask = np.array([[3,2,1],[3,2,1],[3,2,1]])
# image = np.array([[1,2 ,3, 2, 8, 4],
#                   [1, 2, 5, 1, 2, 3],
#                   [2, 8, 4, 1, 2, 5],
#                   [3, 2, 1, 5, 2, 1],
#                   [4, 8, 2, 3, 2, 1],
#                   [1, 2, 5, 2, 8, 4]])
# print(image)
# convolution(mask,image)
mask_average_filter = average_filter(mask)

image_blur_blue = convolution(mask_average_filter,image_blur[:,:,0])
image_blur[:,:,0] = image_blur_blue

image_blur_greeen = convolution(mask_average_filter,image_blur[:,:,1])
image_blur[:,:,1] = image_blur_greeen

image_blur_red = convolution(mask_average_filter,image_blur[:,:,2])
image_blur[:,:,2] = image_blur_red


g_mask = image  - image_blur
image_result = image + (k*g_mask)
cv2.imwrite("image_result_HighBoost_filter.tif",image_result)
cv2.imwrite("image_blur.tif",image_blur)

# print(shift(-1,-1,image))

True